In [ ]:
import pandas as pd
import requests 
import io #File-related I/O operations i.e. file read/write

In [ ]:
url = "https://storage.googleapis.com/uber-data-engineering-project/uber_data.csv"
response = requests.get(url)
df = pd.read_csv(io.StringIO(response.text), sep=',')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

In [ ]:
df[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].info()

In [ ]:
# Drop duplicate and reset index
df = df.drop_duplicates().reset_index(drop=True)

# Assign index to new column
df['trip_id'] = df.index

In [ ]:

df.head()

In [ ]:
df.info()

In [ ]:
# Create new columns for datetime elements
datetime_dim = df[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].reset_index(drop=True)
datetime_dim['pickup_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pickup_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pickup_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday
datetime_dim['pickup_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pickup_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['dropoff_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
datetime_dim['dropoff_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
datetime_dim['dropoff_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday
datetime_dim['dropoff_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
datetime_dim['dropoff_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year

# Create new column 'datetime_id' as index 
datetime_dim['datetime_id'] = datetime_dim.index

# Shift 'datetime_id' column to first position
first_column = datetime_dim.pop('datetime_id')
datetime_dim.insert(loc=0, column='datetime_id', value=first_column)

In [ ]:
datetime_dim.head()

In [ ]:
# Create 'passenger_count_dim' table and reset index
passenger_count_dim = df[['passenger_count']].reset_index(drop=True)

# Create new column 'passenger_count_id' as index 
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index

# Rearrange columns by switching 'passenger_count_id' to first column
passenger_count_dim = passenger_count_dim[['passenger_count_id', 'passenger_count']]

passenger_count_dim.head()

In [ ]:
# Create 'trip_distance_dim' table and reset index
trip_distance_dim = df[['trip_distance']].reset_index(drop=True)

# Create new column 'trip_distance_id' as index 
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index

# Rearrange columns by switching 'passenger_count_id' to first column
trip_distance_dim = trip_distance_dim[['trip_distance_id', 'trip_distance']]

trip_distance_dim.head()

In [ ]:
# Create 'pickup_location_dim' table and reset index
pickup_location_dim = df[['pickup_longitude', 'pickup_latitude']].reset_index(drop=True)

# Create new column 'pickup_location_id' as index 
pickup_location_dim['pickup_location_id'] = pickup_location_dim.index

# Shift 'pickup_location_id' column to first position
first_column = pickup_location_dim.pop('pickup_location_id')
pickup_location_dim.insert(loc=0, column='pickup_location_id', value=first_column)

In [ ]:
# Create 'dropoff_location_dim' table and reset index
dropoff_location_dim = df[['dropoff_longitude', 'dropoff_latitude']].reset_index(drop=True)

# Create new column 'dropoff_location id' as index 
dropoff_location_dim['dropoff_location_id'] = dropoff_location_dim.index

# Rearrange columns by switching 'dropoff_location_id' to first column
dropoff_location_dim = dropoff_location_dim[['dropoff_location_id', 'dropoff_longitude', 'dropoff_latitude']]

dropoff_location_dim.head()

In [ ]:
# Create 'rate_code_type' dictionary
rate_code_type = {
    1:'Standard rate',
    2:'JFK',
    3:'Newark',
    4:'Nassau or Westchester',
    5:'Negotiated fare',
    6:'Group ride'
}

# Create 'rate_code_dim' table and reset index
rate_code_dim = df[['RatecodeID']].reset_index(drop=True)

# Create new column 'dropoff_location id' as index 
rate_code_dim['rate_code_id'] = rate_code_dim.index

# Map 'rate_code_type' values to keys in 'RatecodeID'
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_type)

# Rearrange columns by switching 'rate_code_id' to first column
rate_code_dim = rate_code_dim[['rate_code_id', 'RatecodeID', 'rate_code_name']]

rate_code_dim.head()

In [ ]:
# Create 'payment_type_name' dictionary
payment_type_name = {
    1:'Credit card',
    2:'Cash',
    3:'No charge',
    4:'Dispute',
    5:'Unknown',
    6:'Voided trip'
}

# Create 'payment_type_dim' table and reset index
payment_type_dim = df[['payment_type']].reset_index(drop=True)

# Create new column 'payment_type_id id' as index 
payment_type_dim['payment_type_id'] = payment_type_dim.index

# Map 'payment_type_name' values to keys in 'payment_type'
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_type_name)

# Rearrange columns by switching 'dropoff_location_id' to first column
payment_type_dim = payment_type_dim[['payment_type_id', 'payment_type', 'payment_type_name']]

payment_type_dim.head()

In [ ]:
fact_table = df.merge(passenger_count_dim, left_on='trip_id', right_on='passenger_count_id') \
                .merge(trip_distance_dim, left_on='trip_id', right_on='trip_distance_id') \
                .merge(rate_code_dim, left_on='trip_id', right_on='rate_code_id') \
                .merge(payment_type_dim, left_on='trip_id', right_on='payment_type_id') \
                .merge(datetime_dim, left_on='trip_id', right_on='datetime_id') \
                .merge(pickup_location_dim, left_on='trip_id', right_on='pickup_location_id') \
                .merge(dropoff_location_dim, left_on='trip_id', right_on='dropoff_location_id') \
                [['trip_id', 'VendorID', 'datetime_id', 'passenger_count_id', 'trip_distance_id', 'pickup_location_id', 
                 'dropoff_location_id', 'rate_code_id', 'payment_type_id', 'fare_amount', 'extra',
                 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount']]

fact_table.head()